# Standard packages

In [1]:
import os
import pandas as pd
import time

# Specific installs


In [2]:
method_name = "LinearFold"

In [3]:
!git clone https://github.com/LinearFold/LinearFold.git

Cloning into 'LinearFold'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 422 (delta 100), reused 161 (delta 89), pack-reused 243
Receiving objects: 100% (422/422), 1.29 MiB | 23.65 MiB/s, done.
Resolving deltas: 100% (202/202), done.


In [4]:
import os
os.chdir("LinearFold")

In [5]:
!make

chmod +x linearfold draw_circular_plot
mkdir -p bin
g++ src/LinearFold.cpp -std=c++11 -O3 -Dlv -Dis_cube_pruning -Dis_candidate_list -o bin/linearfold_v 
g++ src/LinearFold.cpp -std=c++11 -O3 -Dis_cube_pruning -Dis_candidate_list -o bin/linearfold_c


# S. cerevisiae (sce) 18 long non-coding RNA dataset
Data source: https://genie.weizmann.ac.il/pubs/PARS10/data/sce_genes_folded.tab.gz

In [6]:
gh_path = "https://raw.githubusercontent.com/sinc-lab/lncRNA-folding/master/data/"
sce = pd.read_csv(gh_path + "sce_genes_folded.tab", delimiter='\t', 
                  header=None, index_col=0, 
                  names=("Gene ID", "sequence", "PARS-assisted folding"))

In [7]:
# Sequences to process
yeast18lnc = ["snR81", "snR34", "snR43", "snR44",  "snR31",  "snR10",
              "snR63", "snR11", "snR82", "snR17b", "snR17a", "snR37",
              "SCR1",  "SRG1",  "snR19", "snR30",  "LSR1",   "TLC1"]

# Compute structures

In [12]:
def run_folding(fasta_name):
  out_file_name = fasta_name + '.dot'

  # Compute structure
  os.system(f"cat {fasta_name} | ./bin/linearfold_v > tmp.dot")

  # Clean output
  out_file_name = "clean_tmp.dot"
  in_lines = open("tmp.dot","r").readlines()
  with open(out_file_name,"w") as out_file:
    for line in in_lines:
      if ">" in line: # extract just the ID
        out_file.write(line.split(":")[1].strip() + "\n")
      else:
        out_file.write(line)

  return out_file_name

In [13]:
out_fasta_name = method_name + "_yeast18"
if os.path.exists(out_fasta_name + ".fasta"): os.remove(out_fasta_name + ".fasta")

lnc_ids = yeast18lnc
print("   \t lnc \t len \t time")
for i, lnc in enumerate(lnc_ids): 

  start_time = time.time()
  seq = sce.loc[lnc]["sequence"]
  print(f"{i+1}/{len(lnc_ids)}\t{lnc} \t {len(seq)}", end='\t')

  # Write a one-sequence fasta
  with open("tmp.fasta", "w") as ofile: 
    ofile.write(f">{lnc}\n{seq}\n")
  
  dot_file_name = run_folding("tmp.fasta")

  # Concatenate outputs
  os.system("cat " + dot_file_name + " >> " + out_fasta_name + ".fasta") 

  print(f"{time.time() - start_time: .1f} s")

!mv LinearFold_yeast18.fasta ../

   	 lnc 	 len 	 time
1/18	snR81 	 201	 0.2 s
2/18	snR34 	 203	 0.2 s
3/18	snR43 	 209	 0.3 s
4/18	snR44 	 211	 0.3 s
5/18	snR31 	 225	 0.3 s
6/18	snR10 	 245	 0.3 s
7/18	snR63 	 255	 0.2 s
8/18	snR11 	 258	 0.2 s
9/18	snR82 	 268	 0.4 s
10/18	snR17b 	 332	 0.5 s
11/18	snR17a 	 333	 0.4 s
12/18	snR37 	 386	 0.5 s
13/18	SCR1 	 522	 0.5 s
14/18	SRG1 	 551	 0.5 s
15/18	snR19 	 568	 0.5 s
16/18	snR30 	 606	 0.6 s
17/18	LSR1 	 1175	 1.4 s
18/18	TLC1 	 1301	 1.5 s
